## Definicion del contexto

El contexto con el que quiero trabajar mi tarea es un proyecto personal en el que me encuentro trabajando el cual corresponde a un bot de trading algorítmico basado en señales. Éste toma estrategias de inversion usando indicadores técnicos y genera señales de compra y venta que ejecuta en los mercados a los que se encuentra conectado mediante API rest o recibiendo datos de manera ágil usando websockets de algunos comercios.

![TradingBot](TradingBot.png)

# Identificacion de requerimientos asociados al caso de uso:
Dentro de los posibles requerimientos asociados al caso de uso tenemos: (Click para abrir cada item)

<details> 
<summary>Problema 1</summary> 

- **Problema:** El bot debe integrarse con múltiples APIs de exchanges (Binance, Coinbase, Kraken), cada una con su propia estructura y formato de datos.
- **Patrón propuesto:** Adapter 
- **Justificación:** Permite adaptar interfaces incompatibles al modelo estándar del bot (`ExchangeClient`), facilitando la integración con múltiples exchanges sin modificar el núcleo de la aplicación. 
</details>

<details> 
<summary>Problema 2</summary> 

- **Problema:** Se desea añadir funcionalidades adicionales a las estrategias de trading (como logging, validación o métricas) sin modificar sus clases base.
- **Patrón propuesto:** Decorator 
- **Justificación:** Permite extender dinámicamente el comportamiento de estrategias envolviéndolas con clases adicionales, manteniendo el principio de abierto/cerrado y mejorando la reutilización.
</details>

<details>
<summary>Problema 3</summary> 

- **Problema:** El bot debe acceder a datos de mercado provenientes de diversas fuentes (API REST, WebSocket, base de datos), y tratarlos como una única fuente unificada. 
- **Patrón propuesto:** Facade 
- **Justificación:** Proporciona una interfaz simplificada para un conjunto complejo de subsistemas de adquisición de datos, ocultando la complejidad y mejorando la experiencia del desarrollador.
</details>

<details>
<summary>Problema 4</summary> 

- **Problema:** Algunas estrategias combinan varias señales (como RSI, MACD y volumen) que deben evaluarse de forma jerárquica o conjunta. 
- **Patrón propuesto:** Composite 
- **Justificación:** Permite tratar señales individuales y compuestas de forma uniforme, lo cual facilita la construcción y evaluación de estrategias complejas a partir de componentes simples. 
</details>

<details>
<summary>Problema 5</summary> 

- **Problema:** Los modelos de IA o configuraciones pesadas del bot se usan repetidamente y requieren optimización de memoria. 
- **Patrón propuesto:** Flyweight 
- **Justificación:** Permite reutilizar instancias compartidas de objetos pesados, reduciendo el consumo de memoria y mejorando el rendimiento general del sistema.
</details>

<details>
<summary>Problema 6</summary> 

- **Problema:** El bot debe enviar notificaciones a distintos canales (email, Telegram, consola, API), pero sin acoplar la lógica de generación de alertas con los medios de salida. 
- **Patrón propuesto:** Bridge 
- **Justificación:** Separa la abstracción de la implementación, permitiendo combinar libremente distintos tipos de notificaciones con múltiples canales sin modificar las clases existentes.
</details>

<details>
<summary>Problema 7</summary> 

- **Problema:** Se necesita controlar y restringir el acceso a ciertos recursos sensibles del bot (como la configuración en tiempo de ejecución o la ejecución de órdenes reales). 
- **Patrón propuesto:** Proxy 
- **Justificación:** Permite interponer una capa de control de acceso, logging o caching antes de delegar las acciones al objeto real, mejorando la seguridad y el control del sistema.
</details>